- Nhận user mới
- Tạo RFM cho user đó (toàn cục)
- Phân cụm user đó
- Tạo RFM cho user đó trên các item của user
- Tạo user-item matrix cho từng cụm
- Đi gợi ý sản phẩm cho user đó
- Tính F1 score

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from scipy.sparse import coo_matrix
from collections import defaultdict
from sklearn.neighbors import NearestNeighbors
import concurrent.futures as cfutures

In [2]:
weight_matrix = pd.read_csv('../data/weight_matrix.csv', index_col=0)
weight_matrix

,Cluster 0,Cluster 1,Cluster 2
R,0.163,0.732,0.142
F,0.236,0.165,0.397
M,0.601,0.103,0.461


In [3]:
def get_weights(label: int):
    '''
        input: nhận label của cụm
        output: series chứa weights của cụm
    '''
    return weight_matrix[f'Cluster {label}'].values

In [4]:
# load mô hình kmean (hoặc fuzzy c mean) đã lưu
with open('../data/kmean_model.pkl', 'rb') as f:
    kmean_model = pickle.load(f)
kmean_model

KMeans(n_clusters=3, random_state=12)

In [5]:
# nhận train data
train = pd.read_csv('../data/train_processed.csv', parse_dates=[0])
rfm_train = pd.read_csv('../data/rfm_train.csv')
# nhận test data
test = pd.read_csv('../data/test_processed.csv', parse_dates=[0])
rfm_test = pd.read_csv('../data/rfm_test.csv')

In [6]:
train_labels = kmean_model.predict(rfm_train.iloc[:, 1:].values)
rfm_train['Cluster Label'] = train_labels

In [7]:
test_labels = kmean_model.predict(rfm_test.iloc[:, 1:].values)
rfm_test['Cluster Label'] = test_labels

# Tạo RFM data (user-item)

In [8]:
recency_unit = 7
snapshot_date = dt.datetime(year=2022, month=5, day=12)

def get_recency(d):
    return (snapshot_date - d.max()).days // recency_unit

# recency: lấy ngày snapshot - ngày mua cuối
# frequency: tổng quantity
# monetary: average số tiền mua

scaler = MinMaxScaler()

def get_rfm_data(dataframe, is_train):
    dataframe = (dataframe
            .groupby(['User number', 'Product Name'])
            .agg({'DateKey': get_recency, 'Quantity': 'count', 'Regular price': 'sum'})
    )
    user_id = dataframe.index
    rfm = dataframe.values
    rfm = scaler.fit_transform(rfm)
    return pd.DataFrame(rfm, index=user_id, columns=['Recency', 'Frequency', 'Moneytary'])

In [9]:
# ui: user item
rfm_train_ui = get_rfm_data(train, True).reset_index()
rfm_test_ui = get_rfm_data(test, False).reset_index()

In [10]:
# tạo mapping giữa user number và cluster label
train_cluster_label = dict(rfm_train.groupby('User number')['Cluster Label'].mean().astype('int'))
test_cluster_label = dict(rfm_test.groupby('User number')['Cluster Label'].mean().astype('int'))

In [11]:
rfm_train_ui['Cluster Label'] = rfm_train_ui['User number'].map(train_cluster_label)
rfm_test_ui['Cluster Label'] = rfm_test_ui['User number'].map(test_cluster_label)

In [12]:
# c = w_r * c_r + w_f * c_f + w_m * c_m
list_weights_train = np.array([get_weights(label) for label in rfm_train_ui['Cluster Label']])
rfm_train_ui['Rating'] = (rfm_train_ui.loc[:, ['Recency', 'Frequency', 'Moneytary']] * list_weights_train).sum(axis=1)

list_weights_test = np.array([get_weights(label) for label in rfm_test_ui['Cluster Label']])
rfm_test_ui['Rating'] = (rfm_test_ui.loc[:, ['Recency', 'Frequency', 'Moneytary']] * list_weights_test).sum(axis=1)

In [13]:
rfm_test_ui

,User number,Product Name,Recency,Frequency,Moneytary,Cluster Label,Rating
0,101000281,Membership_1M,0.947368,0.000000,0.002669,1,0.693749
1,101000282,Membership_1M,0.157895,0.026316,0.005437,1,0.120481
2,101000283,Membership_1M,0.947368,0.000000,0.002669,1,0.693749
3,101000344,Membership_1M,0.210526,0.026316,0.008205,1,0.159293
4,101000391,Membership_1M,1.000000,0.000000,0.002669,1,0.732275
...,...,...,...,...,...,...,...
7058,107000620,Membership_6M,0.000000,0.000000,0.015775,0,0.009481
7059,107000623,Membership_1M,0.000000,0.000000,0.002669,0,0.001604
7060,107000624,Membership_6M,0.000000,0.000000,0.015775,0,0.009481
7061,107000625,Membership_6M,0.000000,0.000000,0.015775,0,0.009481


In [14]:
class CF:
    def __init__(self, rfm, k = 5):
        self.rfm = rfm
        self.k = k
        self.make_ui(self.rfm)
        
    def add(self, rfm):
        self.rfm = pd.concat([self.rfm, rfm])
        self.make_ui(self.rfm)
        
    def make_ui(self, rfm):
        self.items = rfm['Product Name'].unique()
        self.size_item = self.items.shape[0]
        self.item_id = np.arange(self.size_item)
        self.map_item = dict(list(zip(self.items, self.item_id)))
        
        # tạo nơi chứa dữ liệu cho từng cụm
        self.cluster_data = defaultdict(dict)
        
        # lặp qua tất cả cụm
        for label in rfm['Cluster Label'].unique():
            
            # lấy data cho từng cụm
            cluster_rfm = rfm[rfm['Cluster Label'] == label]
            
            # lấy danh sách user duy nhất
            users = cluster_rfm['User number'].unique()
            
            # tạo biến lưu kích cỡ user
            size_user = users.shape[0]
            
            # tạo index cho user
            user_id = np.arange(size_user)
            
            # tạo một ánh xạ giữa user và index
            map_user = dict(list(zip(users, user_id)))
            
            # lấy danh sách index của user và item
            user_indices = cluster_rfm['User number'].map(map_user)
            item_indices = cluster_rfm['Product Name'].map(self.map_item)
            
            ui_matrix = coo_matrix(
                (cluster_rfm.Rating, (user_indices, item_indices)),
                shape=(size_user, self.size_item)
            ).tocsr().toarray()
        
            sim = cosine_similarity(ui_matrix, ui_matrix)
            
            self.cluster_data[label]['rfm'] = cluster_rfm
            self.cluster_data[label]['user_id'] = user_id
            self.cluster_data[label]['map_user'] = map_user
            self.cluster_data[label]['ui_matrix'] = ui_matrix
            self.cluster_data[label]['sim'] = sim
    
    def recommend(self, user_number):
        '''
            nhận vào một user number từ tập dữ liệu test
            sau đó trả về danh sách những sản phẩm được khuyến nghị
        '''
        # tìm cụm mà user đó thuộc về
        cluster_label = None
        for label, data in self.cluster_data.items():
            if user_number in data['map_user'].keys():
                cluster_label = label
                break
                
        cluster_data = self.cluster_data[cluster_label]
        rfm_data = cluster_data['rfm']
        map_user = cluster_data['map_user']
        user_item = cluster_data['ui_matrix']
        similarity_matrix = cluster_data['sim']
        
        # tìm index của user theo id
        user_idx = map_user[user_number]

        list_product_recommendation = []
        # lặp qua tất cả sản phẩm hiện có
        for product_name, product_idx in self.map_item.items():
            bought = False

            # tìm tất cả user đã mua sản phẩm đó
            user_rated_product = rfm_data.query(f'`Product Name` == "{product_name}"')['User number'].values
            user_rated_product = np.array([map_user[i] for i in user_rated_product])

            if user_idx in user_rated_product:
                bought = True
            
            if user_rated_product.shape[0] > 0: 
                # tìm các hệ số tương quan giữa user mong muốn và tất cả user đã rate
                sim = similarity_matrix[user_idx, user_rated_product]
                k_sim = np.argsort(sim)[-self.k:] # chỉ lấy k user tương tự user hiện tại
                # phần k-nearest neighbors

                sim = sim[k_sim]
                user_rated_product = user_rated_product[k_sim]

                # sim = sim[sim < 1] # loại bỏ giá trị có tương quan là 1, vì nó tương quan với chính nó
                rating = user_item[user_rated_product, product_idx] # lấy list rating của những user đã rate

                mean_rating = user_item[user_rated_product].mean(axis=1)
                # tính hệ số rating dự đoán, tổng trọng số giữa rating và độ tương quan
                r = user_item[user_idx].mean() + (sim * (rating - mean_rating)).sum() / (sim.sum() + 1e-8)
            else: # chưa có ai mua sản phẩm này trong cùng một cụm cả
                r = 0

            # đưa vào tuple (product id, rating, bought, is recommend) -> tí nữa sắp xếp giảm dần
            threshold = 0
            list_product_recommendation.append((product_name, r, bought, r > threshold))

        # sắp xếp giảm dần theo rating dự đoán
        list_product_recommendation = sorted(list_product_recommendation, key=lambda x: x[1], reverse=True)
        
        return list_product_recommendation, cluster_label
    
    def __repr__(self):
        return '<CF>'.format(self)

In [15]:
def cal_confusion_matrix(out):
    data = pd.DataFrame(out, columns=['name', 'rating', 'bought', 'recommended'])
    tp = data.query('bought == 1 and recommended == 1').shape[0]
    fp = data.query('bought == 0 and recommended == 1').shape[0]
    fn = data.query('bought == 1 and recommended == 0').shape[0]
    tn = data.query('bought == 0 and recommended == 0').shape[0]
    return tp, fp, fn, tn

In [16]:
# khởi tạo đối tượng CF (collaborative filtering)
# với bộ dữ liệu train
cf = CF(rfm_train_ui, k=1000)

# thêm tập dữ liệu test vào để chuẩn bị test
cf.add(rfm_test_ui)

In [17]:
list_test_user = test['User number'].unique().tolist()

In [18]:
%%time

# quá trình chạy cái này có thể hơi lâu
# 12 phút

def cal(user):
    out, cluster_label = cf.recommend(user)
    return cluster_label, cal_confusion_matrix(out)

with cfutures.ThreadPoolExecutor() as exe:
    list_confusion_matrix = list(exe.map(cal, list_test_user))

Wall time: 9min 53s


In [19]:
# cộng tất cả tn, fp, fn, tn của tất cả user với nhau
metrics = defaultdict(lambda : defaultdict(int))
for cluster_label, (_tp, _fp, _fn, _tn) in list_confusion_matrix:
    metrics[cluster_label]['tp'] += _tp
    metrics[cluster_label]['fp'] += _fp
    metrics[cluster_label]['fn'] += _fn
    metrics[cluster_label]['tn'] += _tn

In [20]:
metrics

defaultdict(<function __main__.<lambda>()>,
            {0: defaultdict(int, {'tp': 39, 'fp': 425, 'fn': 0, 'tn': 667}),
             1: defaultdict(int,
                         {'tp': 7803, 'fp': 171819, 'fn': 0, 'tn': 26415}),
             2: defaultdict(int, {'tp': 509, 'fp': 5420, 'fn': 0, 'tn': 662})})

In [21]:
for label, data in metrics.items():
    precision = data['tp'] / (data['tp'] + data['fp'])
    recall = data['tp'] / (data['tp'] + data['fn'])
    f1_score = (2 * precision * recall) / (precision + recall)
    metrics[label]['precision'] = precision
    metrics[label]['recall'] = recall
    metrics[label]['f1_score'] = f1_score

In [22]:
for label, data in metrics.items():
    print("Label:", label)
    print("Precision:", data['precision'])
    print("Recall:", data['recall'])
    print("F1:", data['f1_score'])
    print("="*50)

Label: 0
Precision: 0.08405172413793104
Recall: 1.0
F1: 0.1550695825049702
Label: 1
Precision: 0.0434412265758092
Recall: 1.0
F1: 0.08326530612244896
Label: 2
Precision: 0.08584921571934559
Recall: 1.0
F1: 0.15812364088226158


In [23]:
def get_recommendation(user_number):
    out, label = cf.recommend(user_number)
    data = pd.DataFrame(out, columns=['name', 'rating', 'bought', 'recommended'])
    return data

In [24]:
# chỗ này đưa ra danh sách dự đoán sản phẩm
# cho một user number nào đó lấy từ tập test
user_number = 101000391
get_recommendation(user_number)

,name,rating,bought,recommended
0,PT 30 session 10tr8,0.597670,False,True
1,PT 10 upgrade to 30 session,0.594654,False,True
2,Membership_1D,0.553393,False,True
3,PT 30 session,0.528977,False,True
4,Membership_1M,0.494398,True,True
5,PT 10 session,0.443642,False,True
6,JoiningFee_2M,0.438639,False,True
7,Membership_3M,0.422918,False,True
8,Water - Revive White 500ml,0.421197,False,True
9,Membership_7M,0.415918,False,True
